# **Análisis de distribución de precios**

In [ ]:
# Módulos

import pandas as pd
import json
import gzip
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

# Carga de datos a DataFrame

Versión no generalizada (Poner tu identificador del dataset), las salidas e interpretaciones tienen que ser cambiadas cuando trabajemos con el dataset entero

In [ ]:
base_dir = Path().resolve().parents[1] # Nos situamos en la carpeta del proyecto
data_dir = base_dir / 'data' # cd data

# No usa variable de entorno pues la ruta será el archivo final con todos los juegos,
# no partes individuales de cada identificador
ruta = Path(data_dir / f'info_steam_games_3.json.gz')

if not ruta.exists():
    raise FileNotFoundError(f'No se encuentra la ruta: {ruta}')

with gzip.open(ruta, 'rt', encoding='UTF-8') as f:
    data = json.load(f)
    
df = pd.DataFrame(data['data'])
df

# Procesado del DataFrame

Dividir cada clave dentro de appdetails en una columna del dataframe

In [ ]:
# Cada clave de appdetails se vuelve una columna del DataFrame
df = df.join(df['appdetails'].apply(pd.Series))
df

- Limpiar columna genres y categories para que solamente salga el campo 'description'. 
- Limpiar columna price_overview para que solamente salga el precio inicial del juego
- Añadir columnas de número de recomendaciones
- Añadir columna de clasificación de juegos por rango de precio

In [ ]:
def get_genres(x):
    if not isinstance(x, dict):
        return []
    genres = x.get('genres', [])
    if not isinstance(genres, list):
        return []
    return [g.get('description') for g in genres if isinstance(g, dict)]

def get_categories(x):
    if not isinstance(x, dict):
        return []
    categories = x.get('categories', [])
    if not isinstance(categories, list):
        return []
    return [g.get('description') for g in categories if isinstance(g, dict)]

def price_range(x):
    if x == 0:
        return 'Free'
    elif x > 0 and x < 20:
        return '[0.01,19.99]'
    elif x >= 20 and x < 40:
        return '[20,39.99]'
    elif x >= 40:
        return '>40'


df['genres'] = df['appdetails'].apply(lambda x: get_genres(x))
df['categories'] = df['appdetails'].apply(lambda x: get_categories(x))
df['price_overview'] = df['price_overview'].apply(lambda x: x.get('initial')/100)
df['price_range'] = df['price_overview'].apply(lambda x: price_range(x))

# df['recomendaciones_positivas'] = df['appreviewhistogram'].apply(lambda x: x.get('rollups').get('recommendations_up') if isinstance(x, dict) & isinstance(x.get('rollups'), dict) else None)
# df['recomendaciones_negativas'] = df['appreviewhistogram'].apply(lambda x: x.get('rollups').get('recommendations_down') if isinstance(x, dict) & isinstance(x.get('rollups'), dict) else None)
# df['recomendaciones_totales'] = df['recomendaciones_negativas'] + df['recomendaciones_positivas']

df.drop(columns=['appdetails', 'appreviewhistogram'], inplace=True, errors='ignore')
df



In [ ]:
df['price_overview'].describe()

Según .describe() se observa que la media de precio en juegos es de 6€ (Esto se verá afectado si en el dataset final eliminamos los juegos f2p), con una desviación tipica de 15.6. (Estos datos cambiaran cuando se use el dataset completo)

<!-- Ahora, vamos a agrupar los juegos por rangos de precio para trabajar más fácilmente con las gráficas -->

# Gráficos + Conclusiones

#### Pie Chart de rango de precios

In [ ]:
plt.figure(figsize=(9,9))
plt.pie(x=df['price_range'].value_counts(),  autopct='%1.1f%%')
plt.legend(df['price_range'].value_counts().index, title='Precios', loc='best')
plt.show()

#### Histograma de precios con transformación logarítmica

In [ ]:
bins_range = np.arange(df['price_overview'].min(), df['price_overview'].max() + 5, 5) # Bins van de 5 en 5 

plt.figure(figsize=(15,15))
plt.hist(x=df['price_overview'], log=True, bins= bins_range)
plt.xticks(np.arange(0, df['price_overview'].max() + 5, 20))
plt.show()

Se puede observar que  mayoría de juegos son tienen un precio menor de 20€, seguido por los juegos gratuitos, juegos con precio menor de 40€ y por último juegos con precio mayor a 40€.

Esto se debe principalmente a que la mayoría de juegos de Steam son públicados bajo desarrolladores Indie, los cuales suelen ser juegos con precio menor de 20€.

Se observa que hay juegos con precio mayor de 100€ e incluso alguno con precio mayor de 500€

- - - 

#### Pie Chart de la distribución de los géneros de cada rango de precios.

In [ ]:
# Por cada genero del array de 'genres', crea una fila por cada miembro del array
df_exploded = df.explode('genres')
df_exploded

# Agrupamos en una tabla los juegos por precio y género
price_genre_counts = df_exploded.groupby(['price_range', 'genres']).size().unstack(fill_value=0)

price_order = ['Free', '[0.01,19.99]', '[20,39.99]', '>40']

fig, ax = plt.subplots(2, 2, figsize=(15, 12))
ax = ax.flatten() 

for ax, price_range in zip(ax, price_order):
    genre_counts = price_genre_counts.loc[price_range]
    genre_counts = genre_counts[genre_counts > 0]
    
    ax.pie(
        genre_counts.values,
        labels=genre_counts.index,
        autopct='%1.1f%%',
    )
    ax.set_title(f'Géneros en rango de precio: {price_range}')
    
plt.tight_layout()
plt.show()

#### Bar plot de los precios y los géneros

In [ ]:
# Agrupamos los juegos en una tabla por género y precio
genre_price_counts = df_exploded.groupby(['genres', 'price_range']).size().unstack(fill_value=0)

genre_price_counts.plot(kind='barh', figsize=(12,8))

plt.xlabel('Número de juegos')
plt.ylabel('Género')
plt.title('Distribución de precio por género')
plt.legend(title='Precio')
plt.tight_layout()
plt.show()


- - -

Conclusión general: Se observa una distribución de cola larga con alta concentración en los juegos menores de 20 € y estas distribuciones se mantienen en cada género.